# Season 17 Election Notes

>**FORBIDDEN KNOWLEDGE WARNING**
>
>This document contains some information that is not currently present on the main site and may be considered spoilers.
>Continue at your own discretion. Some information in this document cannot be shared on the Discord without spoiler tags.

In [74]:
import pandas
%matplotlib inline
from blaseball_mike.models import *
import plotly.express as plot
import plotly.io as _pio
import plotly.subplots as subplot
from IPython.display import display, Markdown

import os
import sys
pdir = os.path.abspath(os.path.join(os.path.dirname(''), os.path.pardir))
sys.path.append(pdir)
from display import *
from blessings import *
sys.path.remove(pdir)

_pio.renderers.default = "notebook_connected"

pies = Team.load_by_name("Philly Pies")

In [75]:
sim = SimulationData.load()
display(Markdown(f"**Last Updated Season {sim.season}, Day {sim.day}**"))

**Last Updated Season 16, Day 116**

---
## Decrees

Decrees are chosen by majority votes across all teams. `3` decrees will pass this season.


All this season's decrees build various new structures that will presumably add gameplay mechanics.

* Forum: Upshelling Events on the Feed will earn you Coins and Prizes.
* Library: Reveal the Ancient Histories of Blaseball.
* Laboratory: Add a Will that lets you build a custom Item and give it to a Player on your Team.
* Smithy: All non-Playoff Teams from this Season build a Smithy in their Ballpark, which occasionally repairs Items.
* Monument: The Top 99 Events of each Season will be Remembered.
* Fairgrounds: All games have a chance of being Prize Matches, in which the winning Team will open an Item Crate.

___
## Wills

Will are selected for each team by raffle. Each team will draw `2` random wills. The bottom 4 teams in the league by
standings will draw `3`. The chosen wills are random, but wills with more votes are more likely to be selected.

### Plunder
**Steal a Player from the Idols board. Send a random Player from your Team back.**

Due to the random nature of the target on our team, this tests the best case (swapping the Best Player on the Idol Board
for a position with our worst player in that position) and the worst case (swapping the Best Player on the Idol Board
for a position with our best player in that position).

**Best Case Swap:**

In [76]:
idols = [x.player for x in Idol.load().values() if x.player.league_team_id != pies]

idols.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_idol = idols[0]
idols.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_idol = idols[0]

worst_batter = sort_lineup(pies, 1)[0]
display(Markdown(f"Batting: Swap {best_batting_idol.name} for {worst_batter.name}"))

pandas.DataFrame(trade_players(pies, worst_batter, best_batting_idol))

Batting: Swap Valentine Games for Yusef Puddles

,Pies
batting_change,+11.1585%
pitching_change,+0.0000%
baserunning_change,+6.8787%
defense_change,-1.8374%


In [77]:
worst_pitcher = sort_rotation(pies, 1)[0]
display(Markdown(f"Pitching: Swap {best_pitching_idol.name} for {worst_pitcher.name}"))

pandas.DataFrame(trade_players(pies, worst_pitcher, best_pitching_idol))


Pitching: Swap Pitching Machine for Tiana Takahashi

,Pies
batting_change,+0.0000%
pitching_change,+16.2700%
baserunning_change,+0.0000%
defense_change,+0.0000%


**Worst Case Swap:**

In [78]:
best_batter = sort_lineup(pies, 1, order="best")[0]
display(Markdown(f"Batting: Swap {best_batting_idol.name} for {best_batter.name}"))

pandas.DataFrame(trade_players(pies, best_batter, best_batting_idol))

Batting: Swap Valentine Games for J-x-n Buckley

,Pies
batting_change,+0.0460%
pitching_change,+0.0000%
baserunning_change,+2.8451%
defense_change,-1.0682%


In [79]:
best_pitcher = sort_rotation(pies, 1, order="best")[0]
display(Markdown(f"Pitching: Swap {best_pitching_idol.name} for {best_pitcher.name}"))

pandas.DataFrame(trade_players(pies, best_pitcher, best_pitching_idol))


Pitching: Swap Pitching Machine for Elvis Figueroa

,Pies
batting_change,+0.0000%
pitching_change,+2.4142%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Equivalent Exchange
**Swap a Player from a Team in the opposite Subleague with a Player from your Team. They must be within 2 combined stars of each other.**

See below for the 10 best swap candidates for a particular player, sorted by highest stats in the applicable position.

In [80]:
wild = Subleague.load("aabc11a1-81af-4036-9f18-229c759ca8a9")
all_wild = []
for team in wild.teams.values():
    all_wild.extend(team.lineup + team.rotation)

In [81]:
all_total_stars = get_total_stars(all_wild)
for player in pies.lineup:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.batting_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20]))
    display(set_heatmap(get_batting_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Eduardo Woodman

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.5
Jessica Telephone,5.5,2.5,4.0,3.0
Jacob Haynes,5.5,3.0,2.0,1.5
Rat Mason,5.0,1.5,2.5,2.5
Math Velazquez,5.0,3.5,4.0,2.5
Comfort Septemberish,5.0,1.0,5.0,2.5
Scores Baserunner,5.0,2.0,3.5,4.5
August Sky,4.5,4.5,2.5,3.0
Baby Triumphant,4.5,2.0,2.5,3.0
Grollis Zephyr,4.5,3.5,4.0,3.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.833060,1.019391,0.846540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Jessica Telephone,1.129590,1.113282,1.216455,1.326294,0.982475,0.215902,1.574396,0.100000,0.919039
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Math Velazquez,0.981884,1.027667,1.067639,0.709586,1.146343,0.395484,0.622834,0.060000,0.682863
Comfort Septemberish,0.978574,1.177996,1.187902,0.375357,0.767285,0.611622,1.105479,0.060000,0.832023
Scores Baserunner,0.975384,1.199816,1.000458,0.666032,0.551777,0.001000,0.503956,0.001000,0.427470
August Sky,0.943651,1.072420,0.913515,0.993340,0.772470,0.268792,0.465689,0.001000,0.460190
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
Grollis Zephyr,0.918341,1.015597,1.159573,0.479638,0.637968,0.634753,0.850535,0.001000,0.379088


####Beasley Day

,Batting,Pitching,Baserunning,Defense
Beasley Day,4.5,3.0,3.5,3.0
Jessica Telephone,5.5,2.5,4.0,3.0
Baby Doyle,5.5,2.5,3.0,4.5
Vessalius Sundae,5.5,3.0,4.0,3.0
Jacob Haynes,5.5,3.0,2.0,1.5
Math Velazquez,5.0,3.5,4.0,2.5
Comfort Septemberish,5.0,1.0,5.0,2.5
Scores Baserunner,5.0,2.0,3.5,4.5
Hiroto Cerna,5.0,4.0,4.0,3.0
August Sky,4.5,4.5,2.5,3.0


...

,Batting,Pitching,Baserunning,Defense
Yusef Puddles,2.0,2.5,3.0,3.5
Jacob Haynes,5.5,3.0,2.0,1.5
Rat Mason,5.0,1.5,2.5,2.5
Baby Triumphant,4.5,2.0,2.5,3.0
Bauer Zimmerman,4.5,1.5,3.0,2.5
Felix Garbage,4.5,3.5,2.0,3.0
Denzel Scott,4.5,2.5,2.5,2.5
Peanutiel Duffy,4.0,3.5,1.5,3.5
Paula Turnip,4.0,1.0,3.5,3.0
Fran Beans,4.0,3.0,2.5,2.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Felix Garbage,0.859852,0.766287,1.174788,0.656731,0.329195,0.001000,1.036772,0.001000,0.680168
Denzel Scott,0.850411,0.963515,0.849616,0.566107,0.627399,0.119743,0.673793,0.007747,0.949707
Peanutiel Duffy,0.841420,1.038584,0.738454,0.996131,0.670202,0.418180,0.343720,0.100000,0.240592
Paula Turnip,0.816815,0.766132,1.017299,0.583585,0.885149,0.544858,0.284335,0.100000,0.261251
Fran Beans,0.785526,0.853002,0.978388,0.568602,0.482162,0.685546,0.313539,0.001000,0.881484


####Usurper Violet

,Batting,Pitching,Baserunning,Defense
Usurper Violet,3.0,3.0,1.5,3.5
Jacob Haynes,5.5,3.0,2.0,1.5
Rat Mason,5.0,1.5,2.5,2.5
Baby Triumphant,4.5,2.0,2.5,3.0
Bauer Zimmerman,4.5,1.5,3.0,2.5
Felix Garbage,4.5,3.5,2.0,3.0
Denzel Scott,4.5,2.5,2.5,2.5
Peanutiel Duffy,4.0,3.5,1.5,3.5
Paula Mason,4.0,3.0,3.5,2.5
Paula Turnip,4.0,1.0,3.5,3.0


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Usurper Violet,0.579715,0.894648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176
Jacob Haynes,1.066191,1.263815,0.942263,1.101913,1.219008,0.001000,0.396483,0.060000,0.823724
Rat Mason,1.009065,1.290628,0.798484,1.077502,0.906716,0.030115,1.092972,0.001000,0.653243
Baby Triumphant,0.927529,0.939663,1.109047,0.626394,0.730501,0.001000,0.217618,0.047784,0.847350
Bauer Zimmerman,0.884063,0.963640,0.967942,0.413869,0.969917,0.426705,0.926229,0.100000,0.225597
Felix Garbage,0.859852,0.766287,1.174788,0.656731,0.329195,0.001000,1.036772,0.001000,0.680168
Denzel Scott,0.850411,0.963515,0.849616,0.566107,0.627399,0.119743,0.673793,0.007747,0.949707
Peanutiel Duffy,0.841420,1.038584,0.738454,0.996131,0.670202,0.418180,0.343720,0.100000,0.240592
Paula Mason,0.827577,0.694852,1.055756,0.534305,0.665276,0.001000,0.875907,0.053370,0.324011
Paula Turnip,0.816815,0.766132,1.017299,0.583585,0.885149,0.544858,0.284335,0.100000,0.261251


In [82]:
for player in pies.rotation:
    total_stars = get_total_stars(player)[player.id]
    valid_total_stars = {k: v for k, v in all_total_stars.items() if total_stars-2 < v < total_stars+2}
    player_matches = [x for x in all_wild if x.id in valid_total_stars.keys()]
    player_matches.sort(key=lambda x: x.pitching_rating, reverse=True)
    display(Markdown(f"####{player.name}"))
    display(get_stars([player] + player_matches[0:20]))
    display(set_heatmap(get_pitching_stlats([player] + player_matches[0:20]), maxVal=1.5))

####Henry Marshallow

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,4.0,2.5,4.5
Patchwork Southwick,4.0,5.5,3.0,4.0
Rafael Davids,3.5,5.0,2.5,4.0
Jayden Wright,3.5,5.0,3.5,4.0
Xandra Pancakes,2.5,5.0,4.0,4.5
Sixpack Santiago,2.0,5.0,4.5,3.0
Liam Snail,3.0,4.5,3.5,4.0
Yummy Elliott,3.5,4.5,4.0,4.0
Cell Barajas,3.5,4.5,2.5,3.5
Howell Rocha,3.0,4.5,3.0,3.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.768310,0.491270,1.171077,1.428524,0.822605,0.808351,0.876106
Patchwork Southwick,1.062257,1.540457,0.697152,1.346364,0.526488,1.388100,0.970742
Rafael Davids,1.024624,0.877302,1.168981,1.132574,1.063992,1.102898,1.225367
Jayden Wright,0.997723,0.941544,1.040374,0.959604,1.202524,0.989975,0.489550
Xandra Pancakes,0.983680,1.082650,1.004702,0.855130,0.854402,0.470938,0.725690
Sixpack Santiago,0.969910,0.765921,1.199201,1.531857,0.747720,0.826116,0.559834
Liam Snail,0.927995,1.032431,0.858471,0.775995,1.083139,1.016481,0.996009
Yummy Elliott,0.912388,0.798342,1.147276,1.108293,0.617898,0.948697,0.932231
Cell Barajas,0.897041,1.000652,1.021738,0.796261,0.660169,0.187258,0.212798
Howell Rocha,0.889888,0.991393,0.917381,0.727428,0.863727,0.538830,0.487715


####Nerd Pacheco

,Batting,Pitching,Baserunning,Defense
Nerd Pacheco,4.5,4.0,4.5,3.0
Cory Twelve,4.5,5.5,4.5,3.0
Patchwork Southwick,4.0,5.5,3.0,4.0
Rafael Davids,3.5,5.0,2.5,4.0
Jayden Wright,3.5,5.0,3.5,4.0
Xandra Pancakes,2.5,5.0,4.0,4.5
Walton Sports,2.5,5.0,4.5,5.5
Sixpack Santiago,2.0,5.0,4.5,3.0
Liam Snail,3.0,4.5,3.5,4.0
Yummy Elliott,3.5,4.5,4.0,4.0


...

,Batting,Pitching,Baserunning,Defense
Elvis Figueroa,4.0,6.0,2.5,5.0
Cory Twelve,4.5,5.5,4.5,3.0
Patchwork Southwick,4.0,5.5,3.0,4.0
Rafael Davids,3.5,5.0,2.5,4.0
Jayden Wright,3.5,5.0,3.5,4.0
Xandra Pancakes,2.5,5.0,4.0,4.5
Walton Sports,2.5,5.0,4.5,5.5
Liam Snail,3.0,4.5,3.5,4.0
Yummy Elliott,3.5,4.5,4.0,4.0
Fitzgerald Blackburn,5.5,4.5,2.5,4.5


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Elvis Figueroa,1.173083,1.217065,1.206718,1.134739,0.851240,0.514752,0.772422
Cory Twelve,1.093782,1.164010,1.189760,0.847783,0.701766,1.191994,0.349467
Patchwork Southwick,1.062257,1.540457,0.697152,1.346364,0.526488,1.388100,0.970742
Rafael Davids,1.024624,0.877302,1.168981,1.132574,1.063992,1.102898,1.225367
Jayden Wright,0.997723,0.941544,1.040374,0.959604,1.202524,0.989975,0.489550
Xandra Pancakes,0.983680,1.082650,1.004702,0.855130,0.854402,0.470938,0.725690
Walton Sports,0.973727,1.259897,0.798052,0.686300,1.135276,0.722438,1.201971
Liam Snail,0.927995,1.032431,0.858471,0.775995,1.083139,1.016481,0.996009
Yummy Elliott,0.912388,0.798342,1.147276,1.108293,0.617898,0.948697,0.932231
Fitzgerald Blackburn,0.894261,0.859707,1.024373,1.000059,0.774466,0.444692,0.794618


####Tiana Takahashi

,Batting,Pitching,Baserunning,Defense
Tiana Takahashi,1.0,3.0,2.5,3.0
Wyatt Mason IV,1.0,4.5,2.0,4.0
Silvia Rugrat,2.5,4.5,1.0,1.5
Alexandria Rosales,3.5,4.5,2.0,2.0
Bennett Bluesky,2.0,4.0,1.0,3.0
Montgomery Bullock,1.5,4.0,2.0,3.5
Mags Banananana,1.5,3.5,3.0,2.5
Emmett Tabby,2.0,3.0,1.0,3.0
Cudi Di Batterino,2.5,3.0,1.5,3.5
Jasper Blather,2.5,3.0,2.5,3.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,0.618850,0.910065,0.961592,0.716677,0.028954,0.596122,0.945940
Wyatt Mason IV,0.909013,0.859159,1.110696,0.885281,0.977619,0.194373,0.879172
Silvia Rugrat,0.884303,0.953588,1.113439,0.507152,0.657504,1.065964,1.218332
Alexandria Rosales,0.850843,1.047870,0.946711,0.989889,0.210874,0.792020,0.787636
Bennett Bluesky,0.754988,1.029258,0.838034,0.275517,0.762323,0.842354,0.850486
Montgomery Bullock,0.754236,1.236754,0.606389,0.554061,0.360671,1.098515,0.411039
Mags Banananana,0.656907,0.970959,0.873893,0.229340,0.430627,0.156140,0.517634
Emmett Tabby,0.642030,0.627873,1.011535,0.576134,0.334005,0.404504,0.948400
Cudi Di Batterino,0.627383,0.708442,0.664072,0.923520,0.314453,0.905493,0.301326
Jasper Blather,0.622595,0.866794,0.705388,0.237008,0.647793,0.872436,0.258707


### Foreshadow
**Swap a Player from your Shadows to your active roster.**

There are 14 shadows players to swap with our 12 existing players. Below is a breakdown comparing the best shadows
players with the worst player in the particular position.

Note that position does not matter for the swap: any shadows player can bat or pitch.


**Batting:**

In [83]:
shadows = pies.bullpen + pies.bench
shadows.sort(key=lambda x: x.batting_rating, reverse=True)
best_batting_shadows = shadows[0:4]
worst_batter = sort_lineup(pies, 1)[0]

display(get_stars([worst_batter] + best_batting_shadows))
display(set_heatmap(get_batting_stlats([worst_batter] + best_batting_shadows), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Yusef Puddles,2.0,2.5,3.0,3.5
Haruta Byrd,3.5,2.5,2.5,1.5
Javier Lotus,3.5,1.0,3.5,4.0
August Obrien,3.5,1.0,3.5,2.0
Doc Anice,3.0,3.5,3.0,4.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Yusef Puddles,0.429562,1.033698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Haruta Byrd,0.724391,0.721433,0.917077,0.853010,0.585921,0.832203,0.169263,0.100000,0.124161
Javier Lotus,0.682621,0.938964,0.386700,0.901328,0.942211,0.205027,0.884526,0.100000,0.815592
August Obrien,0.681669,0.641217,0.927569,0.913891,0.492708,0.923854,0.556849,0.100000,0.463958
Doc Anice,0.620593,0.759195,0.392327,0.792557,0.733223,0.174657,1.058193,0.329574,0.817492


**Pitching:**

In [84]:
shadows.sort(key=lambda x: x.pitching_rating, reverse=True)
best_pitching_shadows = shadows[0:4]
worst_pitcher = sort_rotation(pies, 1)[0]

display(get_stars([worst_pitcher] + best_pitching_shadows))
display(set_heatmap(get_pitching_stlats([worst_pitcher] + best_pitching_shadows), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Tiana Takahashi,1.0,3.0,2.5,3.0
Lucy Tokkan,2.0,3.5,1.0,3.0
Doc Anice,3.0,3.5,3.0,4.5
Twofurious Puddles,2.5,3.5,2.0,3.5
Case Lancaster,0.0,3.5,2.0,3.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Tiana Takahashi,0.618850,0.910065,0.961592,0.716677,0.028954,0.596122,0.945940
Lucy Tokkan,0.741748,0.901966,0.995562,0.263758,0.973020,0.180980,0.321805
Doc Anice,0.734279,1.076739,0.487964,0.959145,0.632001,0.765347,0.305586
Twofurious Puddles,0.694922,0.811110,0.650034,0.807712,0.718615,0.416240,0.725894
Case Lancaster,0.658365,0.867044,0.829197,0.190119,0.825333,0.868251,0.082117


### Move
**Move a Player on your Team to the desired location on your Roster.**

Move would move a player from any location on the team (including Shadows) to another position, increasing the number
of players in that position. See below for moving our best hitting pitcher or best pitching hitter into their preferred
position.

**Batting**

In [85]:
bhp = best_hitting_pitcher(pies)
bph = best_pitching_hitter(pies)

display(Markdown(f"Moving {bhp.name} to the Lineup"))
display(pandas.DataFrame(move_player(pies, bhp, "lineup")))

display(Markdown(f"Moving {worst_batter.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_batter, "bench")))

Moving Nerd Pacheco to the Lineup

,Pies
batting_change,+1.1431%
pitching_change,+0.4324%
baserunning_change,+1.6232%
defense_change,-1.7365%


Moving Yusef Puddles to the Shadows

,Pies
batting_change,+5.3934%
pitching_change,+0.0000%
baserunning_change,+2.2553%
defense_change,-0.0356%


**Pitching**

In [86]:
display(Markdown(f"Moving {bph.name} to the Rotation"))
display(pandas.DataFrame(move_player(pies, bph, "rotation")))

display(Markdown(f"Moving {worst_pitcher.name} to the Shadows"))
display(pandas.DataFrame(move_player(pies, worst_pitcher, "bullpen")))

Moving Ruslan Greatness to the Rotation

,Pies
batting_change,-2.0571%
pitching_change,-0.5950%
baserunning_change,-4.4490%
defense_change,-4.2182%


Moving Tiana Takahashi to the Shadows

,Pies
batting_change,+0.0000%
pitching_change,+7.6747%
baserunning_change,+0.0000%
defense_change,+0.0000%


### Transfuse
**Boost a Player on your Team by 15% to 30% in a Stat Category of your choice, but Impair them in a Random other Category.**

The stat that is negatively impacted is random. It is generally better to boost pitchers, as only the pitching stats are
used for that position. See below for how this would affect various team members at the 2 extreme percentages. This will
only affect one player.

**15% Change:**

In [87]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.15)
display(infuse_lineup_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.15)), maxVal=1.5))

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,4.885272,0.719972
Beasley Day,4.274532,4.998553,0.724021
Huber Frumple,4.287244,5.039095,0.751851
Ruslan Greatness,4.755482,5.439372,0.683890
J-x-n Buckley,6.592812,7.226397,0.633585
Lang Richardson,3.162214,3.907698,0.745485
Yusef Puddles,2.147811,3.068746,0.920935
Usurper Violet,2.898577,3.704516,0.805939


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.977054,1.169391,0.996540,0.800666,0.725519,0.445517,0.724554,0.010000,0.855869
Beasley Day,0.999711,1.195160,1.054890,0.522953,0.697377,0.061371,0.883784,0.010000,1.159991
Huber Frumple,1.007819,0.657458,1.460733,1.082690,1.371501,0.076988,0.835091,0.010000,1.263159
Ruslan Greatness,1.087874,0.772589,1.295689,1.719998,1.620457,0.010000,1.461486,0.010000,0.833552
J-x-n Buckley,1.445279,1.717382,1.192101,1.896709,2.080045,0.010000,2.136793,0.010000,1.543504
Lang Richardson,0.781540,0.962722,0.550621,0.772843,1.091821,0.055246,0.648400,0.010000,0.701015
Yusef Puddles,0.613749,1.183698,0.334577,0.825196,0.301908,0.480778,0.260821,0.010000,1.160015
Usurper Violet,0.740903,1.044648,0.443503,1.172726,1.019795,0.419699,0.433561,0.010000,0.637176


In [88]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.15)
display(infuse_rotation_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.15)), maxVal=1.5))


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.841550,4.774535,0.932985
Nerd Pacheco,4.180600,5.151709,0.971109
Elvis Figueroa,5.865415,6.788955,0.923540
Tiana Takahashi,3.094250,4.393265,1.299014


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.954907,0.641270,1.321077,1.578524,0.972605,0.958351,1.026106
Nerd Pacheco,1.030342,1.237720,1.216351,0.422113,0.758134,1.082212,0.634500
Elvis Figueroa,1.357791,1.367065,1.356718,1.284739,1.001240,0.664752,0.922422
Tiana Takahashi,0.878653,1.060065,1.111592,0.866677,0.178954,0.746122,1.095940


**30% Change:**

In [89]:
infuse_lineup_table, _, _ = improve_team_batting_table(pies, 0.30)
display(infuse_lineup_table)
display(set_heatmap(get_batting_stlats(improve_team_batting(pies, 0.30)), maxVal=1.5))

,old_batting_stars,new_batting_stars,change_in_batting_stars
Eduardo Woodman,4.165300,5.585918,1.420618
Beasley Day,4.274532,5.676802,1.402269
Huber Frumple,4.287244,5.741328,1.454084
Ruslan Greatness,4.755482,6.104166,1.348684
J-x-n Buckley,6.592812,7.852389,1.259577
Lang Richardson,3.162214,4.606233,1.444019
Yusef Puddles,2.147811,3.885398,1.737587
Usurper Violet,2.898577,4.460398,1.561822


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,1.117184,1.319391,1.146540,0.950666,0.875519,0.295517,0.874554,0.010000,1.005869
Beasley Day,1.135360,1.345160,1.204890,0.672953,0.847377,0.010000,1.033784,0.010000,1.309991
Huber Frumple,1.148266,0.807458,1.610733,1.232690,1.521501,0.010000,0.985091,0.010000,1.413159
Ruslan Greatness,1.220833,0.922589,1.445689,1.869998,1.770457,0.010000,1.611486,0.010000,0.983552
J-x-n Buckley,1.570478,1.867382,1.342101,2.046709,2.230045,0.010000,2.286793,0.010000,1.693504
Lang Richardson,0.921247,1.112722,0.700621,0.922843,1.241821,0.010000,0.798400,0.010000,0.851015
Yusef Puddles,0.777080,1.333698,0.484577,0.975196,0.451908,0.330778,0.410821,0.010000,1.310015
Usurper Violet,0.892080,1.194648,0.593503,1.322726,1.169795,0.269699,0.583561,0.010000,0.787176


In [90]:
infuse_rotation_table, _, _ = improve_team_pitching_table(pies, 0.30)
display(infuse_rotation_table)
display(set_heatmap(get_pitching_stlats(improve_team_pitching(pies, 0.30)), maxVal=1.5))


,old_pitching_stars,new_pitching_stars,change_in_pitching_stars
Henry Marshallow,3.841550,5.714489,1.872939
Nerd Pacheco,4.180600,6.110125,1.929525
Elvis Figueroa,5.865415,7.728461,1.863046
Tiana Takahashi,3.094250,5.399357,2.305106


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,1.142898,0.791270,1.471077,1.728524,1.122605,1.108351,1.176106
Nerd Pacheco,1.222025,1.387720,1.366351,0.572113,0.908134,1.232212,0.784500
Elvis Figueroa,1.545692,1.517065,1.506718,1.434739,1.151240,0.814752,1.072422
Tiana Takahashi,1.079871,1.210065,1.261592,1.016677,0.328954,0.896122,1.245940


### Alternate Trust
**Call in an Alternate for a Player on your Team, with a mystery Benefit.**

An Alternate is a player reroll and the Benefit is an unknown modification. Below shows the change in Team Average Stars from rerolling our worst
player, assuming they would be replaced with the Statistically Average player.

In [91]:
worst_pie = sort_overall(pies, 1)[0]

display(Markdown(f"Rerolling {worst_pie.name}"))
if worst_pie in pies.rotation:
    out = replace_player(pies, worst_pie, pitch_star=NEW_PLAYER["pitching"])
else:
    out = replace_player(pies, worst_pie, bat_star=NEW_PLAYER["batting"], run_star=NEW_PLAYER["baserunning"], def_star=NEW_PLAYER["defense"])
pandas.DataFrame(out)

Rerolling Yusef Puddles

,Pies
batting_change,-0.3695%
pitching_change,+0.0000%
baserunning_change,-0.7936%
defense_change,-2.7410%


### Reform
**Re-roll an eligible Permanent Modification for a Player on your Team.**

Below is a list of all players with modifications that could be rerolled. It would be better to reroll modifications
with negative effects.

In [92]:
mod_players = [x for x in pies.lineup + pies.rotation if len(x.perm_attr) > 0]
mod_list = pandas.DataFrame()
for player in mod_players:
    data = pandas.Series({"mods": [x.title for x in player.perm_attr if x.id not in UNREFORMED_MODS]}, name=player.name)
    if len(data["mods"]) == 0:
        continue
    mod_list = mod_list.append(data)
mod_list

,mods
Beasley Day,[Minimalist]
Ruslan Greatness,[Friend of Crows]
J-x-n Buckley,[Fire Eater]
Henry Marshallow,[Maximalist]
Tiana Takahashi,[Triple Threat]


### Enrich
**Pick a modification for your Team next season.**

This will gives the option of choosing between
* The Fourth Strike: Those with the Fourth Strike will get an extra strike in each at bat.
* High Pressure: This Team will play 25% better in Flooding weather when runners are on base.
* Life of the Party: This team gets 10% more from their Party Time stat boosts.
* Home Field Advantage: Teams with Home Field Advantage will start each home game with one run.
* Affinity for Crows: Players with Affinity for Crows will hit and pitch 50% better during Birds weather.
* Traveling: Traveling teams will play 5% better in Away games.

___
## Blessings

Blessings are selected by raffle across all teams. Each blessing will be awarded to one random team, where teams with
more votes are more likely to be selected.

### Buddies
**Two Adjacent Players in your Team's Lineup will gain +15% Moxie.**

Moxie is the attribute most associated with getting walks (plate discipline).
See below for how this would change the attributes of each player.

In [93]:
new_team = []
for player in pies.lineup:
    new_team.append(player.simulated_copy(buffs={"moxie": 0.15}))
display(get_stars(new_team))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,1.5,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
J-x-n Buckley,6.5,3.5,4.5,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.5,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,3.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.847658,1.019391,0.846540,0.650666,0.725519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.870577,1.045160,0.904890,0.372953,0.697377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.864930,0.507458,1.310733,0.932690,1.371501,0.226988,0.685091,0.041371,1.113159
Ruslan Greatness,0.958051,0.622589,1.145689,1.569998,1.620457,0.001000,1.311486,0.001000,0.683552
J-x-n Buckley,1.325985,1.567382,1.042101,1.746709,2.080045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.639492,0.812722,0.400621,0.622843,1.091821,0.205246,0.498400,0.001000,0.551015
Yusef Puddles,0.452270,1.033698,0.184577,0.675196,0.301908,0.630778,0.110821,0.057271,1.010015
Usurper Violet,0.586674,0.894648,0.293503,1.022726,1.019795,0.569699,0.283561,0.100000,0.487176


### Power Chaarge
**Chaarge! Give your Team AA Blood.**

This would make all our players have AA Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### Xenomorph
**Give your Team Acidic Blood.**

This would make all our players have Acidic Blood, and give the team a blood-specific Modification. The effects of this
modification are unknown.

### On Deck
**Call the Alternate for all Players in your Shadows.**

This would randomize all players in our shadows.

### Supply Run
**All Players on your Team with an available Item slot (empty or broken) will open a Generic Jersey Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Rings Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Necklace Crate.**

**All Players on your Team with an available Item slot (empty or broken) will open a Generic Sunglasses Crate.**

For reference, these items boost the following:


| Item Type  | Attributes | Stat |
| ---------- | ---------- | ---- |
| Jersey     | +Musclitude | Batting |
| Ring       | +Ruthlessness | Pitching |
| Necklace   | +Moxie | Batting |
| Sunglasses | +Cinnamon | Vibes |

This would provide Generic Jerseys, Rings, Necklaces, and Sunglasses to the following players:

In [94]:
free_items = [x for x in pies.lineup + pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,1.5,4.0,3.0
Usurper Violet,3.0,3.0,1.5,3.5


### Handcrafted Drop
**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Jersey Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Rings Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Necklace Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Handcrafted Sunglasses Crate.**

For reference, these items boost the following:

| Item Type  | Attributes | Stat |
| ---------- | ---------- | ---- |
| Jersey     | +Musclitude | Batting |
| Ring       | +Ruthlessness | Pitching |
| Necklace   | +Moxie | Batting |
| Sunglasses | +Cinnamon | Vibes |

This would provide Special Jerseys, Rings, Necklaces, and Sunglasses to one of the following players:

**Jersey or Necklace**

In [95]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,1.5,4.0,3.0
Usurper Violet,3.0,3.0,1.5,3.5


**Rings**

In [96]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

""


**Sunglasses**

In [97]:
get_stars(free_items)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,1.5,4.0,3.0
Usurper Violet,3.0,3.0,1.5,3.5


### Crate Drop
**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Dexterity Crate.**

**A Random Player on your Team with an available Item slot (empty or broken) will open a Slime Crate.**

**A Random Hitter on your Team with an available Item slot (empty or broken) will open a Blood Crate.**

**A Random Pitcher on your Team with an available Item slot (empty or broken) will open an Intelligence Crate.**

For reference, these items boost the following:

| Item Type    | Attributes | Stat |
| ------------ | ---------- | ---- |
| Dexterity    | +Thwackability, +Omniscience, +Continuation | Batting, Defense, Baserunning |
| Slime        | ??? | ??? |
| Blood        | ??? | ??? |
| Intelligence | +Ruthlessness, +Anticapitalism, +Shakespearianism | Pitching, Defense |

This will open various crates for a random player on our team.


**Dexterity or Blood**

In [98]:
free_items_lineup = [x for x in pies.lineup if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_lineup)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,1.5,4.0,3.0
Usurper Violet,3.0,3.0,1.5,3.5


**Intelligence**

In [99]:
free_items_rotation= [x for x in pies.rotation if (x.evolution + 1) - len([x for x in x.items if x.health > 0]) > 0]
get_stars(free_items_rotation)

""


**Slime**

In [100]:
get_stars(free_items)

,Batting,Pitching,Baserunning,Defense
Huber Frumple,4.5,1.5,4.0,3.0
Usurper Violet,3.0,3.0,1.5,3.5


### In a Pinch
**Promote the Best Baserunner in your Team's Shadows to a random position in the Lineup.**

See below for new player and the change in team stats.

In [101]:
new_lineup = pies.bench + pies.bullpen
new_lineup.sort(key=lambda x: x.baserunning_rating, reverse=True)
best_shadows_runner = new_lineup[0]

display(Markdown(f"Moving {best_shadows_runner.name} to the Lineup"))
display(get_stars([best_shadows_runner] + pies.lineup))
display(set_heatmap(get_baserunning_stlats([best_shadows_runner] + pies.lineup), maxVal=1))
pandas.DataFrame(move_player(pies, best_shadows_runner, "lineup"))

Moving Mcfarland Vargas to the Lineup

,Batting,Pitching,Baserunning,Defense
Mcfarland Vargas,1.5,1.5,4.0,2.0
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,1.5,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
J-x-n Buckley,6.5,3.5,4.5,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,3.5


,Baserunning Rating,Laserlikeness,Continuation,Base Thirst,Indulgence,Ground Friction
Mcfarland Vargas,0.756649,0.922765,0.557085,0.812168,0.482377,0.421246
Eduardo Woodman,0.703874,0.617191,1.040482,0.526095,0.707616,0.860506
Beasley Day,0.725277,0.619583,0.669022,1.189892,0.592646,0.934967
Huber Frumple,0.808039,0.766723,0.710497,0.853913,0.929982,0.793744
Ruslan Greatness,1.032786,1.375092,0.602818,0.693571,1.108719,0.605824
J-x-n Buckley,0.886173,0.889885,1.392203,0.706722,0.713522,0.762351
Lang Richardson,0.716308,0.742498,0.421602,0.845647,0.446945,0.988950
Yusef Puddles,0.563489,0.607851,0.230396,0.996456,0.455636,0.371812
Usurper Violet,0.334915,0.829727,0.996871,0.222526,0.013191,0.015431


,Pies
batting_change,-5.8923%
pitching_change,+0.0000%
baserunning_change,+0.3921%
defense_change,-3.3620%


### Patchwork
**Promote the Best Defender in your Team's Shadows to a random position in the Lineup.**

See below for new player and the change in team stats.

In [102]:
new_lineup.sort(key=lambda x: x.defense_rating, reverse=True)
best_shadows_defender = new_lineup[0]

display(Markdown(f"Moving {best_shadows_defender.name} to the Lineup"))
display(get_stars([best_shadows_defender] + pies.lineup))
display(set_heatmap(get_defense_stlats([best_shadows_defender] + pies.lineup), maxVal=1))
pandas.DataFrame(move_player(pies, best_shadows_defender, "lineup"))

Moving Doc Anice to the Lineup

,Batting,Pitching,Baserunning,Defense
Doc Anice,3.0,3.5,3.0,4.5
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,1.5,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
J-x-n Buckley,6.5,3.5,4.5,3.5
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,3.5


,Defense Rating,Omniscience,Tenaciousness,Watchfulness,Anticapitalism,Chasiness
Doc Anice,0.862564,0.829694,1.017277,0.954003,0.510123,0.657617
Eduardo Woodman,0.710874,0.439614,0.424310,0.914677,1.014311,1.020885
Beasley Day,0.609757,0.355386,0.398364,1.304772,0.457722,0.593574
Huber Frumple,0.580774,0.472205,0.267753,0.465153,1.016524,0.577598
Ruslan Greatness,1.012064,1.170016,0.894059,0.903316,0.848210,1.344688
J-x-n Buckley,0.657751,0.636443,0.470729,0.438125,0.927871,0.415402
Lang Richardson,0.748934,1.043616,0.940524,0.374060,0.390675,0.394329
Yusef Puddles,0.719280,0.489394,0.972955,0.277902,0.654868,0.898316
Usurper Violet,0.694892,0.964183,0.640791,0.433876,0.290291,0.546039


,Pies
batting_change,-2.0723%
pitching_change,+0.0000%
baserunning_change,-0.8389%
defense_change,+1.6197%


### Alternator
**Recruit a Random Player in the League with the Alternate Modification to your Team.**

See below for list of all living players with Alternate. This will not remove an existing player from our team.

In [103]:
all_players = Player.load_all()
siphons = [x for x in all_players.values() if x.league_team_id is not None and x.deceased is False and "ALTERNATE" in x._perm_attr_ids]
get_stars(siphons)

,Batting,Pitching,Baserunning,Defense
Sexton Wheerer,4.5,4.0,3.5,5.0
Jasmine Washington,2.0,2.5,1.5,2.0
Malik Destiny,5.0,5.5,5.0,4.0
Oscar Vaughan,0.5,1.5,3.0,2.0
Betsy Trombone,2.0,2.0,2.5,2.5
NaN,2.0,1.0,1.5,1.0
Socks Maybe,1.5,0.0,2.5,3.0
Inez Owens,4.0,2.5,3.0,4.0
Evelton McBlase II,1.0,1.5,2.5,3.5
Cory Twelve,4.5,5.5,4.5,3.0


### Strange Attractor
**Recruit a Random Player in the League with the Attractor Modification to your Team.**

See below for list of all living players with Attractor. This will not remove an existing player from our team.
Note: This lists the False Stars, their player attributes are likely different. It is unknown what this means.

In [104]:
all_players = Player.load_all()
siphons = [x for x in all_players.values() if x.league_team_id is not None and x.deceased is False and "ATTRACTOR" in x._perm_attr_ids]
get_stars(siphons)

,Batting,Pitching,Baserunning,Defense
Alx Keming,4.0,1.5,6.0,6.0
Kurt Crueller,7.0,7.5,3.5,7.5
Ren Hunter,4.5,5.0,6.5,5.5
Brisket Friendo,4.0,1.0,4.0,4.0
Peanut Holloway,6.5,4.5,5.5,6.0
Alyssa Harrell,6.0,6.5,4.5,6.0
Chorby Short,5.0,7.5,6.5,5.5
Nagomi Mcdaniel,5.0,4.5,7.0,6.0
Liquid Friend,6.0,3.5,4.5,4.5
Forrest Best,6.0,9.0,7.0,7.0


### Last Not Least
**The Last Three Players in your Team's Lineup will gain 10% Thwackability.**

Thwackability is the attribute most associated with making contact.
See below for how this would change the attributes of each player.

In [105]:
new_team = pies.lineup[0:-3]
for player in pies.lineup[-3:]:
    new_team.append(player.simulated_copy(buffs={"thwackability": 0.1}))
display(get_stars(new_team))
display(set_heatmap(get_batting_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
Huber Frumple,4.5,1.5,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
J-x-n Buckley,6.5,3.5,4.5,3.5
Lang Richardson,3.5,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,3.5


,Batting Rating,Thwackability,Divinity,Musclitude,Moxie,Patheticism,Martyrdom,Tragicness,Buoyancy
Eduardo Woodman,0.833060,1.019391,0.846540,0.650666,0.575519,0.595517,0.574554,0.100000,0.705869
Beasley Day,0.854906,1.045160,0.904890,0.372953,0.547377,0.211371,0.733784,0.001000,1.009991
Huber Frumple,0.857449,0.507458,1.310733,0.932690,1.221501,0.226988,0.685091,0.041371,1.113159
Ruslan Greatness,0.951096,0.622589,1.145689,1.569998,1.470457,0.001000,1.311486,0.001000,0.683552
J-x-n Buckley,1.318562,1.567382,1.042101,1.746709,1.930045,0.001000,1.986793,0.001000,1.393504
Lang Richardson,0.658658,0.912722,0.400621,0.622843,0.941821,0.205246,0.498400,0.001000,0.551015
Yusef Puddles,0.443672,1.133698,0.184577,0.675196,0.151908,0.630778,0.110821,0.057271,1.010015
Usurper Violet,0.601618,0.994648,0.293503,1.022726,0.869795,0.569699,0.283561,0.100000,0.487176


### Set the Table
**Move the Best Overall Hitter in your Team's Lineup or Rotation to the #3 spot in your Team's Lineup.**

See below for the new lineup order.

In [106]:
new_lineup = pies.lineup.copy()
new_lineup.sort(key=lambda x: x.batting_rating, reverse=True)
best_divinity = new_lineup[0]

new_lineup = pies.lineup.copy()
new_lineup.insert(2, new_lineup.pop(new_lineup.index(best_divinity)))
get_stars(new_lineup)

,Batting,Pitching,Baserunning,Defense
Eduardo Woodman,4.0,2.0,3.5,3.5
Beasley Day,4.5,3.0,3.5,3.0
J-x-n Buckley,6.5,3.5,4.5,3.5
Huber Frumple,4.5,1.5,4.0,3.0
Ruslan Greatness,5.0,4.0,5.0,5.0
Lang Richardson,3.0,0.5,3.5,3.5
Yusef Puddles,2.0,2.5,3.0,3.5
Usurper Violet,3.0,3.0,1.5,3.5


### Grounded
**A random player in your Team's Rotation will gain 40% Overpowerment but lose 20% Unthwackability.**

Overpowerment is the attribute most associated with reducing big hits (Homers, Triples).
Unthwackability is the attribute most associated with reducing hits in general.
See below for how this would change the attributes of each player.

In [107]:
new_team = []
for player in pies.rotation:
    new_team.append(player.simulated_copy(buffs={"overpowerment": 0.4, "unthwackability": -0.2}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,3.0,2.5,4.5
Nerd Pacheco,4.5,4.5,4.5,3.0
Elvis Figueroa,4.0,5.5,2.5,5.0
Tiana Takahashi,1.0,3.0,2.5,3.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,0.613912,0.291270,1.171077,1.828524,0.822605,0.808351,0.876106
Nerd Pacheco,0.865071,0.887720,1.066351,0.672113,0.608134,0.932212,0.484500
Elvis Figueroa,1.122062,1.017065,1.206718,1.534739,0.851240,0.514752,0.772422
Tiana Takahashi,0.584237,0.710065,0.961592,1.116677,0.028954,0.596122,0.945940


### Untouchable
**A random player in your Team's Rotation will gain 40% Unthwackability but lose 20% Overpowerment.**

Overpowerment is the attribute most associated with reducing big hits (Homers, Triples).
Unthwackability is the attribute most associated with reducing hits in general.
See below for how this would change the attributes of each player.

In [108]:
new_team = []
for player in pies.rotation:
    new_team.append(player.simulated_copy(buffs={"overpowerment": -0.2, "unthwackability": 0.4}))
display(get_stars(new_team))
display(set_heatmap(get_pitching_stlats(new_team), maxVal=1))

,Batting,Pitching,Baserunning,Defense
Henry Marshallow,3.5,5.0,2.5,4.5
Nerd Pacheco,4.5,4.0,4.5,3.0
Elvis Figueroa,4.0,6.5,2.5,5.0
Tiana Takahashi,1.0,3.5,2.5,3.0


,Pitching Rating,Unthwackability,Ruthlessness,Overpowerment,Shakespearianism,Coldness,Suppression
Henry Marshallow,1.011709,0.891270,1.171077,1.228524,0.822605,0.808351,0.876106
Nerd Pacheco,0.801236,1.487720,1.066351,0.072113,0.608134,0.932212,0.484500
Elvis Figueroa,1.313423,1.617065,1.206718,0.934739,0.851240,0.514752,0.772422
Tiana Takahashi,0.706936,1.310065,0.961592,0.516677,0.028954,0.596122,0.945940


---
## Appendix
* [Description of Attributes](https://www.blaseball.wiki/w/Player_Attributes)
* [Stlat Viewer](https://slavfox.space/abslve/?foreboding-kaleidoscope#PHIL)
* [Historical Player Graphs](http://yoori.space/hloroscopes/)
